# Tutorial #4: Provision a feature store in various ways

In this tutorial you will use various approaches to provision a feature store.
- Create a feature store using CLI/SDK
  - Default experience of creating a feature store
  - Create a feature store using provided storage as feature store default blob storage
  - Create a feature store using provided storage as offline store
  - Create a feature store using provided Redis cluster as online store
  - Create a feature store using provided UAI as the managed identity used for feature store materialization job
- Create a feature store using ARM template
- Create a feature store using Bicep template


## Prerequisites and Setup

- Create a compute instance
- Run this notebook using above created compute instance and choose `Python 3.8 - AzureML` kernel

### Setup root directory for the samples

In [ ]:
import os

root_dir = "../../"

if os.path.isdir(root_dir):
    print("The folder exists.")
else:
    print("The folder does not exist. Please create or fix the path")

### Setup resource top directory

In [ ]:
subscription_id = "your-subscription-id"
resource_group_name = "your-resource-group-name"
location = "eastus2"

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import FeatureStore
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group_name,
)

#### Setup CLI

1. Install AzureML CLI extention and azure-ai-ml package
1. Authenticate
1. Set the default subscription

In [ ]:
!az extension add --name ml

In [ ]:
!pip install azure-ai-ml

In [ ]:
# Authenticate
!az login

In [ ]:
!az account set -s $subscription_id

## 1 Create a feature store using CLI/SDK
### 1.1 Default feature store creation experience

With this default feature store provision experience, you will:

- Provide the subscription id, resource group name and location where you want to provision your feature store.
- Provide the feature store name.

Then, the system will provision following resources:

(1) provision a storage account with storage type as `Azure Data Lake Storage Gen2` that used as workspace default storage account

(2) create a container in the above storage as offline store.

(3) provision an user managed identity(UAI) used for feature store materialization job.

(4) provision the RBAC of UAI to feature store and to offline store.

    Below is the details RBAC permission the system helps to grant:

|Scope|	Action/Role|
|--|--|
|Feature store | `AzureML Data Scientist` role|
|Container as feature store offline store | `Storage Blob Data Contributor` role|
|Storage account as workspace default storage account	|`Storage Blob Data Contributor` role|

(5) create an offline store connection to the feature store workspace

(6) provision other resources for workspace including keyvault etc.

In [ ]:
featurestore_name = "myfs-sdk"

fs = FeatureStore(
    name=featurestore_name,
    location=location,
)

# wait for feature store creation
fs_poller = ml_client.feature_stores.begin_create(fs)

print(fs_poller.result())

Alternatively, you can dump the feature store configuration to an Yaml file and provision the feature store using CLI.

In [ ]:
featurestore_name = "myfs-cli"

yaml_path = root_dir + "/featurestore/featurestore_default.yaml"
fs.dump(yaml_path)

!az ml feature-store create --name $featurestore_name --subscription $subscription_id --resource-group $resource_group_name --file $yaml_path

### 1.2 Create a feature store with provided storage as workspace default storage

Create a storage account that will be used as workspace default storage account

In [ ]:
featurestore_name = "myfswithblob"

In [ ]:
storage_account_name = featurestore_name + "storage1"
!az storage account create --name $storage_account_name --enable-hierarchical-namespace true --resource-group $resource_group_name --location $location --subscription $subscription_id

In [ ]:
# Specify storage account
storage_account = f"/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}/providers/Microsoft.Storage/storageAccounts/{storage_account_name}"

fs = FeatureStore(
    name=featurestore_name,
    location=location,
    storage_account=storage_account,
)

# wait for feature store creation
fs_poller = ml_client.feature_stores.begin_create(fs)
print(fs_poller.result())

Alternatively, you can dump the feature store configuration to an Yaml file and use CLI to provision it.

In [ ]:
yaml_path = root_dir + "/featurestore/featurestore_with_blob.yaml"
fs.dump(yaml_path)

featurestore_name = "myfswithblobcli"

!az ml feature-store create --name $featurestore_name --subscription $subscription_id --resource-group $resource_group_name --file $yaml_path

### 1.3 Create a feature store with provided storage as the offline store
User can bring your own storage as the offline store storage for feature store.

In [ ]:
featurestore_name = "myfswithoffline"

In [ ]:
offline_storage_account_name = featurestore_name + "offline1"

!az storage account create --name $offline_storage_account_name --enable-hierarchical-namespace true --resource-group $resource_group_name --location $location --subscription $subscription_id

In [ ]:
offline_store_container_name = "offlinestore"
connection_string = "your connection string to your storage"

!az storage fs create --name $offline_store_container_name --account-name $offline_storage_account_name --subscription $subscription_id --connection-string f"{connection_string}"

In [ ]:
from azure.ai.ml.entities import MaterializationStore

gen2_container_arm_id = f"/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}/providers/Microsoft.Storage/storageAccounts/{offline_storage_account_name}/blobServices/default/containers/{offline_store_container_name}"

offline_store = MaterializationStore(
    type="azure_data_lake_gen2", target=gen2_container_arm_id
)

fs = FeatureStore(
    name=featurestore_name,
    location=location,
    offline_store=offline_store
)
# wait for feature store creation
fs_poller = ml_client.feature_stores.begin_create(fs)
print(fs_poller.result())

Alternatively, you can dump the feature store configuration to an Yaml file and use CLI to provision it.

In [ ]:
yaml_path = root_dir + "/featurestore/featurestore_with_offline.yaml"
fs.dump(yaml_path)

featurestore_name = "myfswithofflinecli"

!az ml feature-store create --name $featurestore_name --subscription $subscription_id --resource-group $resource_group_name --file $yaml_path

### 1.4 Create a new Redis instance as online store

In the following code cell, define the name of the Azure Cache for Redis that you want to create or reuse. Optionally, you can override other default settings.

In [ ]:
redis_name = "your-redis-cluster-name"
sku = "premium"
size = "P2"

You can select the Redis cache tier (basic, standard, premium, or enterprise). You should choose a SKU family that is available for the selected cache tier. See this documentation page to learn more about [how selecting different tiers may affect cache performance](https://learn.microsoft.com/azure/azure-cache-for-redis/cache-best-practices-performance). See this link learn more about [pricing for different SKU tiers and families of Azure Cache for Redis](https://azure.microsoft.com/en-us/pricing/details/cache/).

Execute the following code cell to create an Azure Cache for Redis with premium tier, SKU family P and cache capacity 2. It may take approximately 5-10 minutes to provision the Redis instance.

In [ ]:
!az redis create --name $redis_name --resource-group $resource_group_name --location $location --sku $sku --vm-size $size

In [ ]:
featurestore_name = "myfs-redis"

In [ ]:
# Specify online store
redis_arm_id = f"/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}/providers/Microsoft.Cache/Redis/{redis_name}"
online_store = MaterializationStore(type="redis", target=redis_arm_id)

fs = FeatureStore(
    name=featurestore_name,
    location=location,
    online_store=online_store,
)

# wait for feature store creation
fs_poller = ml_client.feature_stores.begin_create(fs)
print(fs_poller.result())

Alternatively, you can dump the feature store configuration to an Yaml file and use CLI to provision it.

In [ ]:
yaml_path = root_dir + "/featurestore/featurestore_with_online.yaml"
fs.dump(yaml_path)

featurestore_name = "myfs-redis-cli"

!az ml feature-store create --name $featurestore_name --subscription $subscription_id --resource-group $resource_group_name --file $yaml_path

### 1.5 Create a feature store using provided UAI

In the following code cell, provide a name for the user-assigned managed identity that you would like to create.

In [ ]:
uai_name = "myfsuai"

Execute the following cell to create the UAI.

In [ ]:
!az identity create --subscription $subscription_id --resource-group $resource_group_name --location $location --name $uai_name

##### Retrieve UAI properties
The following code retrieves principal ID, client ID, and ARM ID property values for the created UAI.

In [ ]:
from azure.mgmt.msi import ManagedServiceIdentityClient
from azure.identity import DefaultAzureCredential

msi_client = ManagedServiceIdentityClient(
    DefaultAzureCredential(), subscription_id
)
managed_identity = msi_client.user_assigned_identities.get(
    resource_name=uai_name, resource_group_name=resource_group_name
)

uai_principal_id = managed_identity.principal_id
uai_client_id = managed_identity.client_id
uai_arm_id = f"/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}/providers/Microsoft.ManagedIdentity/userAssignedIdentities/{uai_name}"

In [ ]:
from azure.ai.ml.entities import ManagedIdentityConfiguration

featurestore_name = "myfs-uai"

materialization_identity = ManagedIdentityConfiguration(
    client_id=uai_client_id,
    principal_id=uai_principal_id,
    resource_id=uai_arm_id
)

fs = FeatureStore(
    name=featurestore_name,
    location=location,
    materialization_identity=materialization_identity,
)

# wait for feature store creation
fs_poller = ml_client.feature_stores.begin_create(fs)
print(fs_poller.result())

Alternatively, you can dump the feature store configuration to an Yaml file and use CLI to provision it.

In [ ]:
yaml_path = root_dir + "/featurestore/featurestore_with_uai.yaml"
fs.dump(yaml_path)

featurestore_name = "myfs-uai-cli"

!az ml feature-store create --name $featurestore_name --subscription $subscription_id --resource-group $resource_group_name --file $yaml_path

## 2 Create a feature store using ARM template

Provision a feature store is a complicated process as we provision several resources. We recommend you provision a feature store using CLI/SDK with configuration fits your scenario as described above, then download the deployment template, update the parameters, and deploy the ARM template directly.

- Provision a feature store, my-featurestore, using CLI.
- The CLI command will output the deployment url, and the deployment is named as my-featurestore-xxxxxxx. Click on the deployment link.
- From the deployment link page, click on the `Template` tab from the left panel.
- From the `Template` page, click on the `Download` button from the top menu.
- You will get two Json files, `template.json` and `parameters.json`.
- Update the `parameters.json` file with appropriate parameter values.
- Run this command to deploy: `az deployment group create --name {your-deployment-name} --resource-group {your-resource-group} --template-file template.json --parameters parameters.json`
- Run this command to check deployment status: `az deployment group show --name {your-deployment-name}`

Under `featurestore/arm-template` folder, you will find an example ARM template. This is the template generated by provisioning a default feature store. We go through the `parameters.json` file to explain the parameters for several resources that is important to feature store. 

### 2.1 Feature store section

This section describes feature store name, resource group, location. Please update this section for your feature store. Note, feature store is a `featurestore` kind of workspace.
```json
"parameters": {
        "workspaceName": {
            "value": "my-featurestore"
        },
        "description": {
            "value": "my-featurestore"
        },
        "friendlyName": {
            "value": "my-featurestore"
        },
        "kind": {
            "value": "featurestore"
        },
        "location": {
            "value": "eastus2"
        },
        "resourceGroupName": {
            "value": "my-resource-group"
        }
}
```

### 2.2 Default storage Section

This section describes the default blob storage used for the feature store workspace. Note, the `storageAccountOption` value is `new`, which means the system will provision a new storage with the name and resource group etc. provided. You can also use an existing storage. In this case, you need to set `storageAccountOption` to `existing`, and provide the storage name and resource group.

```json
"parameters": {
        "storageAccountOption": {
            "value": "new"
        },
        "storageAccountName": {
            "value": "myfeaturestorestorage4116d3e469"
        },
        "storageAccountType": {
            "value": "Standard_LRS"
        },
        "storageAccountBehindVNet": {
            "value": "false"
        },
        "storageAccountResourceGroupName": {
            "value": "my-resource-group"
        },
        "storageAccountLocation": {
            "value": "eastus2"
        }
}
```

### 2.3 Key vault section

System will provision a new key vault, and the name is usually `feature store name + keyvault + random string`. Note, the `KeyVaultName` cannot have any special characters.
```json
"parameters": {
        "keyVaultOption": {
            "value": "new"
        },
        "keyVaultName": {
            "value": "myfeaturestorekeyvaultf7a63f0f7"
        },
        "keyVaultBehindVNet": {
            "value": "false"
        },
        "keyVaultResourceGroupName": {
            "value": "my-resource-group"
        },
        "keyVaultLocation": {
            "value": "eastus2"
        }
}
```

### 2.4 Application Insight section
System will provision a new Application Insight for this feature store. You can replace `myfeaturestore` to any name you like, and add some random string for the resource name.

```json
"parameters": {
        "applicationInsightsOption": {
            "value": "new"
        },
        "logAnalyticsName": {
            "value": "myfeaturestorelogalyti09a8f09c8"
        },
        "logAnalyticsArmId": {
            "value": "/subscriptions/my-subscription-id/resourceGroups/my-resource-group/providers/Microsoft.OperationalInsights/workspaces/myfeaturestorelogalyti09a8f09c8"
        },
        "applicationInsightsName": {
            "value": "myfeaturestoreinsights4684a3029"
        },
        "applicationInsightsResourceGroupName": {
            "value": "my-resource-group"
        },
        "applicationInsightsLocation": {
            "value": "eastus2"
        }
}
```


### 2.5 Offline store section

In this example, the `offline_store_storage_account_name` is same as the one for default blob storage. The system will create a container named `myfeaturestorecontaine8ebdffb55` inside this storage to use as the offline store. You will also need to update the `offline_store_connection_name` with a random string at the end for connection name.


```json
"parameters": {
        "offlineStoreStorageAccountOption": {
            "value": "new"
        },
        "offline_store_storage_account_name": {
            "value": "myfeaturestorestorage4116d3e469"
        },
        "offline_store_container_name": {
            "value": "myfeaturestorecontaine8ebdffb55"
        },
        "offline_store_resource_group_name": {
            "value": "my-resource-group"
        },
        "offline_store_subscription_id": {
            "value": "my-subscription-id"
        },
        "offline_store_connection_name": {
            "value": "OfflineStoreConnectionName-b4bb8712"
        }
}
```

### 2.6 Managed identity section

System will provision a user managed identity to be used for feature store materialization job. You can choose to grant this identity neccessary permission to feature store and offline store by setting `grant_materialization_permissions` to `true`.

```json
"parameters": {
        "materializationIdentityOption": {
            "value": "new"
        },
        "materialization_identity_name": {
            "value": "materialization-uai-dbebceefeb2c331aab47f9c1a4716666"
        },
        "materialization_identity_subscription_id": {
            "value": "my-subscription-id"
        },
        "materialization_identity_resource_group_name": {
            "value": "my-resource-group"
        },
        "grant_materialization_permissions": {
            "value": "true"
        }
}
```

### 2.7 Online store section

If you want to configure an online store, you can provide the Redis cluster information as follows. This example assumes you have a Redis cluster named `my-redis-name`.

```json
"parameters": {
        "online_store_resource_id": {
            "value": "/subscriptions/my-subscription-id/resourceGroups/my-resource-group/providers/Microsoft.Cache/Redis/my-redis-name"
        },
        "online_store_resource_group_name": {
            "value": "my-resource-group"
        },
        "online_store_subscription_id": {
            "value": "my-subscription-id"
        },
        "online_store_connection_name": {
            "value": "OnlineStoreConnectionName-42ace817"
        }
}
```

### 2.7 Feature store spark section

This section is specific to feature store workspace.

```json
"parameters": {
        "spark_runtime_version": {
            "value": "3.2.0"
        }
}
```

### 2.8 Default storage vs offline store storage

Feature store workspace has a default storage, which will hold your artifacts such as models and logs etc. Feature store workspace also has an offline storage which hold feature values. You can share the storage. As in above example, the default feature store provision experience is the system create a new data lake gen2 storage which is used as the default blob storage. In addition, the system will create a container to hold feature values.

If you choose to provide an existing storage as the default blob storage for feature store, the system will create a new data lake gen2 storage and a container to be used as the offline store.

If you choose to provide an existing data lake gen2 container as the offline store, the system will create a new storage for use as the default storage (unless you also provide an existing storage for it).


#### 2.8.1 User provide an existing storage as the default blob storage

```json
"parameters": {
        "storageAccountOption": {
            "value": "existing"
        },
        "storageAccountName": {
            "value": "myexistingstorage4116d3e469"
        },
        "storageAccountType": {
            "value": "Standard_LRS"
        },
        "storageAccountBehindVNet": {
            "value": "false"
        },
        "storageAccountResourceGroupName": {
            "value": "my-resource-group"
        },
        "storageAccountLocation": {
            "value": "eastus2"
        },
        "offlineStoreStorageAccountOption": {
            "value": "new"
        },
        "offline_store_storage_account_name": {
            "value": "myfeaturestorestorage4116d3e469"
        },
        "offline_store_container_name": {
            "value": "myfeaturestorecontaine8ebdffb55"
        },
        "offline_store_resource_group_name": {
            "value": "my-resource-group"
        },
        "offline_store_subscription_id": {
            "value": "my-subscription-id"
        },
        "offline_store_connection_name": {
            "value": "OfflineStoreConnectionName-b4bb8712"
        }
}
```

#### 2.8.2 User provide an existing storage container as offline store

```json
"parameters": {
        "storageAccountOption": {
            "value": "new"
        },
        "storageAccountName": {
            "value": "myfeaturestorestorage4116d3e469"
        },
        "storageAccountType": {
            "value": "Standard_LRS"
        },
        "storageAccountBehindVNet": {
            "value": "false"
        },
        "storageAccountResourceGroupName": {
            "value": "my-resource-group"
        },
        "storageAccountLocation": {
            "value": "eastus2"
        },
        "offlineStoreStorageAccountOption": {
            "value": "existing"
        },
        "offline_store_storage_account_name": {
            "value": "myexistingstorage4116d3e469"
        },
        "offline_store_container_name": {
            "value": "myexistingcontaine8ebdffb55"
        },
        "offline_store_resource_group_name": {
            "value": "my-resource-group"
        },
        "offline_store_subscription_id": {
            "value": "my-subscription-id"
        },
        "offline_store_connection_name": {
            "value": "OfflineStoreConnectionName-b4bb8712"
        }
}
```

#### 2.8.3 User provide an existing storage for default blob storage and container for offline store respectively

```json
"parameters": {
        "storageAccountOption": {
            "value": "existing"
        },
        "storageAccountName": {
            "value": "myexistingstorage1"
        },
        "storageAccountType": {
            "value": "Standard_LRS"
        },
        "storageAccountBehindVNet": {
            "value": "false"
        },
        "storageAccountResourceGroupName": {
            "value": "my-resource-group"
        },
        "storageAccountLocation": {
            "value": "eastus2"
        },
        "offlineStoreStorageAccountOption": {
            "value": "existing"
        },
        "offline_store_storage_account_name": {
            "value": "myexistingstorage2"
        },
        "offline_store_container_name": {
            "value": "myexistingcontaine8ebdffb55"
        },
        "offline_store_resource_group_name": {
            "value": "my-resource-group"
        },
        "offline_store_subscription_id": {
            "value": "my-subscription-id"
        },
        "offline_store_connection_name": {
            "value": "OfflineStoreConnectionName-b4bb8712"
        }
}
```

## Clean up

Use following command to delete the feature stores you provisioned. Note, this command use `--all-resources` to clean up all related resources provisioned.

In [ ]:
featurestore_name = "myfs"
!az ml feature-store delete --subscription $subscription_id --resource-group $resource_group_name --name $featurestore_name --all-resources -y